## Data Collection - Players

---

In [1]:
import pandas as pd
from functools import reduce

from client.models.player import Player
from client.scrapers.fifa import get_player_by_name

from client.scrapers.whoscored.player import crawl_player_overall_stats, crawl_player_defensive_stats, \
crawl_player_passing_stats, crawl_player_offensive_stats

In [2]:
league_team_df = pd.read_csv('./league/Premier League.csv')

In [3]:
players_df = pd.DataFrame()
for idx, row in league_team_df.iterrows():
    
    # excute player data crawling functions 
    player_overall_df = crawl_player_overall_stats(int(row.team_id))
    player_defensive_df = crawl_player_defensive_stats(int(row.team_id))
    player_passing_df = crawl_player_passing_stats(int(row.team_id))
    player_offensive_df = crawl_player_offensive_stats(int(row.team_id))

    # merge dataframes
    players_df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'), [player_overall_df,
                        player_defensive_df, player_passing_df, player_offensive_df])

InvalidSessionIdException: Message: Tried to run command without establishing a connection
